In [4]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from functions import *




2022-11-07 16:33:14.900848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Computational Tool Final Project**


In [2]:
clean_books1 = pd.read_csv('generalized_books_and_ratings.csv')
clean_books1 = clean_books1[["Title","description","authors","publishedDate","categories","averageRating"]]
clean_books1['clean_summary'] = clean_books1['description'].apply(lambda x: clean_books(x))
#clean_books1['clean_authors'] = clean_books1['authors'].apply(lambda x: clean_books(x))
clean_books1['clean_categories'] = clean_books1['categories'].apply(lambda x: clean_books(x))
clean_books1 = clean_books1.dropna()
books = clean_books1[["Title" ,"authors" ,"publishedDate" , "clean_summary" , "clean_categories" , "averageRating"] ]
books
books.to_csv("final.csv")




In this project we are going to use a cleaned up version of this data set from Kaggle (https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=books_data.csv&fbclid=IwAR3H0BlyYyiNodMoF0GbzknzNMyriDa_9eL00g1if_Y29QBPcobfeHQpZCE).

This dataset contains two csv files, one containg the ratings and another containg the books. Post cleanup there are 44285 books with  columns 
,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
,Title,description,authors,publishedDate,categories,ratingsCount
* id 
* book name 
* genre 
* summary

The second csv file contains the reviews for the books and has 1502433 reviews total after cleanup (removing the ratings for the books we removed in the cleanup of the first csv file). The columns it has is:
,Id,Title,Price,review/helpfulness,review/score,review/time,review/summary,review/text
,Id,Title,Price,User_id,review/score

Our main goal is to classify the books with text simmilarity ( using summaries) and predict their genre correctly. Moreover, we want to make a simple recomendation UI using github pages which the user can choose between time, rating , genre and get some recomendation based on our classification model.


In [8]:

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
file1 = pd.read_csv("final.csv") 
for index, row in file1["clean_summary"].iteritems():
    if  isinstance(row, str):
        cat_arr = row.split(" ")
        for word in cat_arr:
            if word in stop_words:
                cat_arr.remove(word)
        file1['clean_summary'].loc[index] = cat_arr
    
        
        


    
    






/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_14236/906595094.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file1['clean_summary'].loc[index] = cat_arr


In [10]:
file1

,Unnamed: 0,Title,authors,publishedDate,clean_summary,clean_categories,averageRating
0,0,The Church of Christ: A Biblical Ecclesiology ...,['Everett Ferguson'],1996,"[church, christ, biblical, ecclesiology, today...",religion,4.50
1,1,Voices from the Farm: Adventures in Community ...,['Rupert Fike'],2012-08-21,"[twenty, five, years, ago, height, counter, cu...",biography autobiography,4.82
2,2,The Battleship Bismarck,['Stefan Draminski'],2018-09-20,"[bismarck, perhaps, famous, notorious, warship...",history,3.50
3,3,Tess and the Highlander,['May Mcgoldrick'],2002-11,"[on, windswept, isle, of, scotland, seventeen,...",fiction,4.24
4,4,Beginner's Yoruba (Hippocrene Beginner's Series),['Kayode J. Fakinlede'],2005,"[, beginners, yoruba, now, available, two, acc...",education,3.80
...,...,...,...,...,...,...,...
41401,41830,Thin Within,"['Judy Halliday', 'Arthur Halliday']",2005-04-17,"[want, lose, weight, dieting, doesnt, seem, wo...",health fitness,4.93
41402,41831,The Awakening and Selected Stories (Modern Lib...,['Kate Chopin'],2003,"[it, first, appeared, awakening, greeted, crie...",love stories,3.00
41403,41832,Final things,['Jenny Offill'],2015-03-17,"[graces, father, believes, science, builds, da...",fiction,4.00
41404,41833,The Orphan Of Ellis Island (Time Travel Advent...,['Elvira Woodruff'],2000-06-01,"[school, trip, ellis, island, dominick, avaro,...",fiction,4.68


In [11]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(clean_books1["clean_summary"])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

<IPython.core.display.Javascript object>

/Users/stef_cm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


: 

: 

In [24]:
file1.dtypes

Unnamed: 0            int64
Title                object
authors              object
publishedDate        object
clean_summary        object
clean_categories     object
averageRating       float64
dtype: object

In [25]:
file1['clean_summary'].isna().sum()

0